Work straight from matrix equations

In [9]:
import numpy as np
from cmm_error_map import data_cmpts as dc

In [10]:
p = dc.model_parameters_test.copy()

In [11]:
xyz  = np.array([100.,50.,20.])
x,y,z = xyz
xyzt = np.array([-100.,0.,0.])
xt, yt, zt =xyzt

In [12]:
# linear dependence
ld_dict = {
    "Txx": x,
    "Txy": x,
    "Txz": x,
    "Tyx": y,
    "Tyy": y,
    "Tyz": y,
    "Tzx": z,
    "Tzy": z,
    "Tzz": z,
    "Rxx": x,
    "Rxy": x,
    "Rxz": x,
    "Ryx": y,
    "Ryy": y,
    "Ryz": y,
    "Rzx": z,
    "Rzy": z,
    "Rzz": z,
    "Wxy": x,
    "Wxz": y,
    "Wyz": z,
}

In [13]:
pl = {key: p[key]* ld_dict[key] for key in p}
pl

{'Txx': 0.00133,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': -0.00056,
 'Tyy': -0.0002545,
 'Tyz': 0.0,
 'Tzx': 0.00052,
 'Tzy': 9.2e-05,
 'Tzz': 6.68e-07,
 'Rxx': 7.489999999999999e-07,
 'Rxy': 1.5399999999999999e-06,
 'Rxz': 5e-07,
 'Ryx': -2.2900000000000003e-07,
 'Ryy': -7.15e-07,
 'Ryz': 1.095e-06,
 'Rzx': 4.9799999999999996e-08,
 'Rzy': -1.588e-08,
 'Rzz': 9.56e-07,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [14]:
# rotation
rxl = np.array([[1.,       pl['Rxz'], -pl['Rxy']], 
               [-pl['Rxz'],   1.,      pl['Rxx']],
               [ p['Rxy'],-p['Rxx'],     1.   ]])
ryl = np.array([[1.,       pl['Ryz'], -pl['Ryy']], 
               [-pl['Ryz'],   1.,      pl['Ryx']],
               [ pl['Ryy'],-pl['Ryx'],     1.   ]])
rzl = np.array([[1.,       pl['Rzz'], -pl['Rzy']], 
               [-pl['Rzz'],   1.,      pl['Rzx']],
               [ pl['Rzy'],-pl['Rzx'],     1.   ]])

In [15]:
# translation
xl = np.array([x + pl['Txx'], pl['Txy'], pl['Txz'] ])
yl = np.array([pl['Tyx'], y + pl['Tyy'], pl['Tyz'] ])
zl = np.array([pl['Tzx'], pl['Tzy'], z + pl['Tzz'] ])

In [16]:
# inverse rotation
inv_ryl = np.linalg.inv(ryl)
inv_rzl = np.linalg.inv(rzl)

In [17]:
inv_ryl

array([[ 1.00000000e+00, -1.09499984e-06, -7.15000251e-07],
       [ 1.09500016e-06,  1.00000000e+00,  2.28999217e-07],
       [ 7.14999749e-07, -2.29000783e-07,  1.00000000e+00]])

In [25]:
w = rxl @ yl - rxl @ xl + rxl @ inv_ryl @ zl + rxl @ inv_ryl @ inv_rzl @ xyzt
w

array([-200.0013901 ,   49.99975196,   19.99992413])

In [19]:
from cmm_error_map import design_matrix_linear_fixed as design

In [20]:
design.model_linear(x, y, z, list(p.values()), xt, yt, zt)

(0.0013785000000000002, -0.00013100000000000004, 3.630600000000001e-05)

In [24]:
xyz

array([100.,  50.,  20.])

In [21]:
w - xyz

array([-3.00001390e+02, -2.48038825e-04, -7.58741369e-05])

In [22]:
m = np.array([[1, 2, 3], [1, 2, 3], [1, 2, 3]])
v = np.array([1, 1, 1])
m @ v

array([6, 6, 6])

In [23]:
m

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [30]:
def linear_dependency(xyz, model_params):
    x,y,z = xyz
    ld_dict = {
        "Txx": x,
        "Txy": x,
        "Txz": x,
        "Tyx": y,
        "Tyy": y,
        "Tyz": y,
        "Tzx": z,
        "Tzy": z,
        "Tzz": z,
        "Rxx": x,
        "Rxy": x,
        "Rxz": x,
        "Ryx": y,
        "Ryy": y,
        "Ryz": y,
        "Rzx": z,
        "Rzy": z,
        "Rzz": z,
        "Wxy": x,
        "Wxz": y,
        "Wyz": z,
    }
    pl = {key: p[key]* ld_dict[key] for key in model_params}
    return pl

In [31]:
def linear_model_matrix(xyz, xyzt, model_params):
    pl = linear_dependency(xyz, model_params)
    # rotation
    rxl = np.array([[1.,       pl['Rxz'], -pl['Rxy']], 
                [-pl['Rxz'],   1.,      pl['Rxx']],
                [ p['Rxy'],-p['Rxx'],     1.   ]])
    ryl = np.array([[1.,       pl['Ryz'], -pl['Ryy']], 
                [-pl['Ryz'],   1.,      pl['Ryx']],
                [ pl['Ryy'],-pl['Ryx'],     1.   ]])
    rzl = np.array([[1.,       pl['Rzz'], -pl['Rzy']], 
                [-pl['Rzz'],   1.,      pl['Rzx']],
                [ pl['Rzy'],-pl['Rzx'],     1.   ]])
    # inverse rotation
    inv_ryl = np.linalg.inv(ryl)
    inv_rzl = np.linalg.inv(rzl)
    # translation
    xl = np.array([x + pl['Txx'], pl['Txy'], pl['Txz'] ])
    yl = np.array([pl['Tyx'], y + pl['Tyy'], pl['Tyz'] ])
    zl = np.array([pl['Tzx'], pl['Tzy'], z + pl['Tzz'] ])
    w = rxl @ yl - rxl @ xl + rxl @ inv_ryl @ zl + rxl @ inv_ryl @ inv_rzl @ xyzt
    return w
    
    
    

In [32]:
w = linear_model_matrix(xyz, xyzt, dc.model_parameters_test)
w

array([-200.0013901 ,   49.99975196,   19.99992413])

In [33]:
w = linear_model_matrix(xyz, xyzt, dc.model_parameters_dict)
w

array([-200.0013901 ,   49.99975196,   19.99992413])

In [34]:
dc.model_parameters_dict

{'Txx': 0.0,
 'Txy': 0.0,
 'Txz': 0.0,
 'Tyx': 0.0,
 'Tyy': 0.0,
 'Tyz': 0.0,
 'Tzx': 0.0,
 'Tzy': 0.0,
 'Tzz': 0.0,
 'Rxx': 0.0,
 'Rxy': 0.0,
 'Rxz': 0.0,
 'Ryx': 0.0,
 'Ryy': 0.0,
 'Ryz': 0.0,
 'Rzx': 0.0,
 'Rzy': 0.0,
 'Rzz': 0.0,
 'Wxy': 0.0,
 'Wxz': 0.0,
 'Wyz': 0.0}

In [ ]:
model_params = dc.model_parameters_dict

pl = {key: p[key]* ld_dict[key] for key in model_params}
# rotation
rxl = np.array([[1.,       pl['Rxz'], -pl['Rxy']], 
            [-pl['Rxz'],   1.,      pl['Rxx']],
            [ p['Rxy'],-p['Rxx'],     1.   ]])
ryl = np.array([[1.,       pl['Ryz'], -pl['Ryy']], 
            [-pl['Ryz'],   1.,      pl['Ryx']],
            [ pl['Ryy'],-pl['Ryx'],     1.   ]])
rzl = np.array([[1.,       pl['Rzz'], -pl['Rzy']], 
            [-pl['Rzz'],   1.,      pl['Rzx']],
            [ pl['Rzy'],-pl['Rzx'],     1.   ]])
# inverse rotation
inv_ryl = np.linalg.inv(ryl)
inv_rzl = np.linalg.inv(rzl)
# translation
xl = np.array([x + pl['Txx'], pl['Txy'], pl['Txz'] ])
yl = np.array([pl['Tyx'], y + pl['Tyy'], pl['Tyz'] ])
zl = np.array([pl['Tzx'], pl['Tzy'], z + pl['Tzz'] ])
w = rxl @ yl - rxl @ xl + rxl @ inv_ryl @ zl + rxl @ inv_ryl @ inv_rzl @ xyzt